In [16]:
from __future__ import print_function

# import sys
import sys, os, copy
os.environ['TERM'] = 'linux'
import numpy as np  
import random
random.seed(1)
import time
import pickle
import logging

start_time = time.time()

sys.path.append("/Users/laurengreenspan/fastjet-install/lib/python3.8/site-packages")
sys.path.append("/Users/laurengreenspan/GitDLs/TreeNiNNew/code_ginkgo")
sys.path.append("/Users/laurengreenspan/GitDLs/ginkgo/src/")
sys.path.append("/Users/laurengreenspan/GitDLs/ClusterTrellis/src")
from ClusterTrellis import HierarchicalTrellis
from ClusterTrellis.trellis_node import TrellisNode
import fastjet as fj

from ginkgo import likelihood_invM as likelihood

import preprocess_functions as pf
from ClusterTrellis.utils import get_logger
logger = get_logger(level=logging.WARNING)

In [2]:
in_dir = '/Users/laurengreenspan/GitDLs/TreeNiNNew/code_ginkgo/data/raw_trees'
out_dir = '/Users/laurengreenspan/GitDLs/TreeNiNNew/code_ginkgo/data/reclustered_trees/'

samplename = 'ginkgo'
subjetlist = [filename for filename in np.sort(os.listdir(in_dir)) if (samplename in filename and filename.endswith('.pkl'))]
print(subjetlist)
ifile = 0
file=in_dir+'/'+subjetlist[ifile]
with open(file, "rb") as f: jets_file =pickle.load(f) 

print(len(jets_file))
print(jets_file[0].keys())

['ginkgo_jets0.pkl']
20
dict_keys(['root_id', 'tree', 'content', 'LambdaRoot', 'Lambda', 'Delta_0', 'pt_cut', 'algorithm', 'deltas', 'draws', 'leaves', 'M_Hard', 'logLH', 'dij', 'ConstPhi', 'PhiDelta', 'PhiDeltaRel', 'label', 'pt', 'energy', 'eta', 'mass', 'phi'])


In [3]:
with open('/Users/laurengreenspan/GitDLs/TreeNiNNew/code_ginkgo/data/reclustered_trees/ginkgo_truth_20jets.pkl', "rb") as f:
    test_dict=pickle.load(f)
print(test_dict[0][0].keys())

with open('/Users/laurengreenspan/GitDLs/TreeNiNNew/code_ginkgo/data/preprocessed_trees/test_ginkgo_truth_20jets.pkl', "rb") as f:
    test_dict2=list(pickle.load(f))
test_dict2[0][0].keys()



dict_keys(['root_id', 'tree', 'content', 'LambdaRoot', 'Lambda', 'Delta_0', 'pt_cut', 'algorithm', 'deltas', 'draws', 'leaves', 'M_Hard', 'logLH', 'dij', 'ConstPhi', 'PhiDelta', 'PhiDeltaRel', 'label', 'pt', 'energy', 'eta', 'mass', 'phi'])


dict_keys(['root_id', 'tree', 'content', 'LambdaRoot', 'Lambda', 'Delta_0', 'pt_cut', 'algorithm', 'deltas', 'draws', 'leaves', 'M_Hard', 'logLH', 'dij', 'ConstPhi', 'PhiDelta', 'PhiDeltaRel', 'label', 'pt', 'energy', 'eta', 'mass', 'phi'])

In [10]:
class ModelNode(TrellisNode):
    """Class to define the node pairwise splitting energy and calculate node features for a given model """

    def __init__(self,
                 model_params,
                 elements = None,
                 children = None,
                 map_features = None):
        TrellisNode.__init__(self, model_params, elements, children, map_features)

    def get_energy_of_split(self, a_node, b_node):
        """Model energy function.
        Args: sibling nodes
        returns: model pairwise splitting energy
        """

        logger.debug(f"computing energy of split: {a_node, b_node}")
 
        split_llh = likelihood.split_logLH_with_stop_nonstop_prob(a_node.map_features,
                                           b_node.map_features,
                                           self.model_params["delta_min"],
                                           self.model_params["lam"])
        logger.debug(f"split_llh = {split_llh}")


        return split_llh


    def compute_map_features(self, a_node, b_node):
        """Auxiliary method to get the parent vertex features. This is model dependent. In Ginkgo these are the momentum and parent         invariant mass.
        Args: sibling nodes
        returns: list where each entry is a parent feature, e.g. [feature1,feature2,...]
        """
        momentum = a_node.map_features + b_node.map_features
        logger.debug(f"computing momentum for {a_node, b_node, momentum}")

        return momentum

def runTrellisOnly(gt_trees, NleavesMin, NleavesMax, MaxNjets,pt_cut, lam):
    
    """Create and fill the trellis for a given set of leaves
    Args: 
    gt_trees : truth level  trees.
    NleavesMin: minimum number of leaves to select  trees
    NleavesMax: maximum number of leaves to select  trees
    MaxNjets: maximum number of trees to run the trellis code.
    
    Returns:
    results: dictionary with the following {key:values}
                Z: partition functions,
                trellis_MLE: MAP hierarchy, 
                RunTime: time for each tree
                totJets": total number of trees
                gt_llh: truth level trees llh
 
    """
    
    """ Keep only trees with leaves between NleavesMin and NleavesMax"""
    print(len(gt_trees[1]["leaves"]))
    smallJetIndex =[i for i,gt_tree in enumerate(gt_trees)  if NleavesMin<=len(gt_tree["leaves"])<NleavesMax]
            
    
    results = {"Z":[], "trellis_MLE":[], "RunTime":[], "Ntrees":[]}
    
    if len(smallJetIndex)>0:
        
        GT_trees = np.asarray(gt_trees)[smallJetIndex]
        
        """ Total number of trees to run the trellis"""
        totTrees = np.min([MaxNjets,len(smallJetIndex)])

        results["totTrees"] = totTrees
        results["Nleaves"] = [len(GT_trees[m]["leaves"]) for m in range(totTrees)]
        results["gt_llh"]= [np.sum(GT_trees[m]["logLH"]) for m in range(totTrees)]
        for m in range(totTrees):

#             if m%1000==0:
#                 print("Creating trellis for jet #",m)
                
            startTime = time.time()
            
            data_params = GT_trees[m]
            N=len(data_params['leaves'])
            
            """Replace with current model parameters"""
#             leaves_features =[ [data_params['leaves'][i],0] for i in range(N)]
            leaves_features =[ data_params['leaves'][i] for i in range(N)]
            model_params ={}
#             model_params["delta_min"] = float(data_params['pt_cut'])
#             model_params["lam"]= float(data_params['Lambda'])
            model_params["delta_min"] = float(pt_cut)
            model_params["lam"]= float(lam)

    
            """ Create trellis"""
            trellis = HierarchicalTrellis.HierarchicalTrellis()
            trellis.create_from_leaves(leaves_features, 
                                                          model_params, 
                                                          NodeClass = ModelNode)

            """Compute MAP (MLE), partition function Z"""
            map_energy, Z , Ntrees= np.float64(trellis.compute_map_tree())
            
#             """ Create and fill the trellis"""
#             trellis, Z, map_energy, Ntrees,  totTime = rpe.compare_map_gt_and_bs_trees(GT_trees[m], 
#                                                                                                                                          ModelNode,
#                                                                                                                                          leaves_features,
#                                                                                                                                          model_params)

            results["Z"].append(Z)
            results["trellis_MLE"].append(map_energy)
            results["Ntrees"].append(Ntrees)

            endTime = time.time() - startTime
            results["RunTime"].append(endTime)

    else:
        print("There are no jets in the dataset with the required number of leaves ")

    return results, trellis



In [11]:
jetdef_tree = 'TrellisMAP'

element = jets_file[1]


#smallJetIndex =[i for i,gt_tree in enumerate(gt_trees)  if NleavesMin<=len(gt_tree["leaves"])<NleavesMax]

element




{'root_id': 0,
 'tree': array([[ 1,  2],
        [-1, -1],
        [-1, -1]]),
 'content': array([[633.1547994 , 365.4627204 , 365.4627204 , 365.4627204 ],
        [395.53295407, 231.95317222, 227.38888073, 225.66764705],
        [237.62184533, 133.50954818, 138.07383967, 139.79507334]]),
 'LambdaRoot': tensor(4.7855),
 'Lambda': tensor(4.7855),
 'Delta_0': tensor(196.),
 'pt_cut': 36.0,
 'algorithm': 'truth',
 'deltas': array([196.,   0.,   0.]),
 'draws': array([None, tensor(0.1800), tensor(0.1648)], dtype=object),
 'leaves': array([[395.53295407, 231.95317222, 227.38888073, 225.66764705],
        [237.62184533, 133.50954818, 138.07383967, 139.79507334]]),
 'M_Hard': 14.0,
 'logLH': array([-3.20771119,  0.        ,  0.        ]),
 'dij': [],
 'ConstPhi': [1.0404130548451846, 1.0588952885166083],
 'PhiDelta': [-2.12825263874033],
 'PhiDeltaRel': [3.1755560632595774],
 'label': 0,
 'pt': array([324.819915  , 192.06557384]),
 'energy': 633.1547993974301,
 'eta': 0.48112981192630455,
 'm

In [27]:
minLeaves =1
maxLeaves=30
MaxNjets = 20000

rate = element['Lambda']
pt_cut = element['pt_cut']

"""Find MAP hierarchy and partition function """
times=[]
MAP = []
Zfunc=[]
for i in range(len(jets_file)):
    
    """Dataset"""
    data_params = jets_file[i]
    
    """Number of elements"""
    N=len(data_params['leaves'])

    """Replace with leaves features for current model"""
#     leaves_features =[ [data_params['leaves'][i],0] for i in range(N)]
    leaves_features =[ data_params['leaves'][i] for i in range(N)]
    
    """Replace with current model parameters
    """
    model_params ={}
    model_params["delta_min"] = float(data_params['pt_cut'])
    model_params["lam"]= float(data_params['Lambda'])
    model_params["lam_root"]= float(data_params['LambdaRoot'])
    model_params["N_const"] = N
    
    startTime = time.time()
    
    """ Create trellis"""
    a_trellis = HierarchicalTrellis.HierarchicalTrellis()
    a_trellis.create_from_leaves(leaves_features, 
                                                  model_params, 
                                                  NodeClass = ModelNode)
    
    """Compute MAP (MLE), partition function Z"""
    map_energy, Z , Ntrees= np.float64(a_trellis.compute_map_tree())
    endTime = time.time() - startTime
    
    times.append(endTime)
    MAP.append(map_energy)
    Zfunc.append(Z)

# #model_params = {"delta_min": pt_cut, "lam": rate}
# results, trellis =  runTrellisOnly(jets_file, 
#                                  NleavesMin =minLeaves, 
#                                  NleavesMax= maxLeaves, 
#                                  MaxNjets = MaxNjets,
#                                            pt_cut = pt_cut,
#                                            lam = rate)

In [23]:
MAP
Zfunc

[-9.84420934825374,
 -3.2077111644650724,
 -9.514220786464742,
 -3.358722430871073,
 -9.883279638519772,
 -3.238391645356472,
 -9.529445048264844,
 -9.767029197909686,
 -3.307831051630968,
 -3.2824615763429805,
 -10.432204995774374,
 -3.7163408174093244,
 -9.700131373360865,
 -3.543719774709482,
 -3.7171552796064695,
 -3.616873908601793,
 -9.834511341444504,
 -3.898191021577816,
 -10.310867738405252,
 -3.6846637463595453]

In [ ]:


N_analysis=len(subjetlist)

  jet_pT=[]
  jet_mass=[]
  
  #Loop over all the events
  for element in jets_file:
   if jetdef_tree =='truth':
      jet = element
      label = jet['label']
      reclustered_jets.append((jet, label))
   elif jetdef_tree == 'ptdesc':
      jet = pf.sequentialize_by_pt(element,reverse=True)
      label = jet['label']
      reclustered_jets.append((jet, label))
   elif jetdef_tree == 'ptasc':
      jet = pf.sequentialize_by_pt(element,reverse=False)
      label = jet['label']
      reclustered_jets.append((jet, label))
   else:
      event=pf.make_pseudojet(element['leaves'])
#     print('Event const=',event)
      label=element['label']
#     print('label=',label)
    # Recluster jet constituents
      out_jet = pf.recluster(event, Rjet,jetdef_tree)  
    # #Create a dictionary with all the jet tree info (topology, constituents features: eta, phi, pT, E, muon label)
      jets_tree = pf.make_tree_list(out_jet)
#     print('jets_tree=',jets_tree)
    
    #Keep only the leading jet
      for tree, content, mass, pt in [jets_tree[0]]:
#       print('Content=',content)
         jet_pT.append(pt)
         jet_mass.append(mass)
         jet = pf.make_dictionary(tree,content,mass,pt)
    
#       print('jet dictionary=',jet)
         reclustered_jets.append((jet, label))
#       pickle.dump((jet, label), out_file, protocol=2)
      
      counter+=1

#     print('===='*20)
#     print('===='*20)
    
#     if counter>0:
#       break


#------------------------------------------------------------------------------
# print('Max jet_mass_difference with rot=',np.sort(jet_mass_difference)[-50::])
myNjets = len(reclustered_jets)
print('number of reclustered_jets=',myNjets)

#   out_filename = str(out_dir)+'tree_'+subjetlist[ifile].split('.')[0]+'_'+str(counter)+'.pkl'
out_filename = str(out_dir)+'ginkgo_'+str(jetdef_tree)+'_'+str(myNjets)+'jets.pkl'


# SAVE OUTPUT FILE
print('out_filename=',out_filename)
with open(out_filename, "wb") as f: pickle.dump(reclustered_jets, f, protocol=2) 
    


print('counter=',counter)  
# sys.exit()





In [ ]:

   
import sys
sys.path.append("./ClusterTrellis/src")

import os
import pickle
import string
import time
import logging
import numpy as np
import importlib
import argparse

from ClusterTrellis import run_experiments as rpe
from ClusterTrellis import HierarchicalTrellis
from ClusterTrellis.trellis_node import TrellisNode

from ClusterTrellis.utils import get_logger
logger = get_logger(level=logging.WARNING)

"""Replace with model auxiliary scripts to calculate the energy function"""
#from ClusterTrellis import Ginkgo_likelihood as likelihood
from ginkgo import likelihood_invM as likelihood




